
# 🛍️ AI-Powered Email Assistant for Fashion Store

### Project Overview

This proof-of-concept application leverages Large Language Models (LLMs) with Retrieval-Augmented Generation (RAG) to intelligently process customer emails for a fashion store. The system classifies emails, processes product orders, and generates human-like responses based on a live product catalog.

### Key Features

* **Email Classification**: Categorizes incoming emails as *product inquiries* or *order requests*.
* **Order Processing**: Checks stock availability, updates inventory, and tracks fulfillment status.
* **Response Generation**:

  * Creates personalized replies for successful or failed order requests.
  * Handles product inquiries by retrieving relevant catalog information using vector search.
* **LLM Utilization**: Employs GPT-4o for natural language understanding and generation.
* **Output Format**: Results are exported to a multi-sheet spreadsheet with:

  * `email-classification`
  * `order-status`
  * `order-response`
  * `inquiry-response`



# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

In [ ]:
#Import dependencies
import numpy as np
import pandas as pd
from openai import OpenAI
import json
from typing import List, Dict, Tuple
import re
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe
from google.auth import default
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# Code example of OpenAI communication
from openai import OpenAI

client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='<openai_api_key>'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Classify emails

In [ ]:
class EmailClassifier:
    def __init__(self, api_key: str, base_url: str):
        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key
        )

    def _create_classification_prompt(self, email: Dict) -> str:
        """Create a detailed prompt for email classification."""
        return f"""You are an expert email classifier for a fashion retail business. Your task is to classify the following email as either a "product inquiry" or an "order request".

        Email Details:
        Subject: {email['subject']}
        Message: {email['message']}

        Classification Rules:
        1. "order request" if:
          - Contains specific product IDs or names
          - Mentions quantities to purchase
          - Uses words like "order", "buy", "purchase", "send me"
          - Contains payment or shipping information
          - Expresses intent to make a purchase

        2. "product inquiry" if:
          - Asks about product details, features, or specifications
          - Seeks recommendations or advice
          - Asks about availability or stock
          - Contains questions about products
          - Seeks information without clear purchase intent

        Please analyze the email and respond with ONLY one of these two categories: "product inquiry" or "order request".
        Do not include any explanations or additional text in your response."""

    def classify_email(self, email: Dict) -> str:
        """Classify a single email using GPT-4."""
        prompt = self._create_classification_prompt(email)

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert email classifier for a fashion retail business."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,  # Low temperature for consistent results
            max_tokens=10     # We only need a short response
        )

        classification = response.choices[0].message.content.strip().lower()
        return "product inquiry" if "inquiry" in classification else "order request"

    def classify_emails(self, emails_df: pd.DataFrame) -> pd.DataFrame:
        """Classify all emails in the dataframe."""
        classifications = []

        for _, row in emails_df.iterrows():
            email = {
                'subject': row['subject'],
                'message': row['message']
            }
            classification = self.classify_email(email)
            classifications.append(classification)

        # Create result dataframe
        result_df = pd.DataFrame({
            'email ID': emails_df['email_id'],
            'category': classifications
        })

        return result_df

def setup_google_sheets():
    """Set up Google Sheets authentication and create output document."""
    # Authenticate with Google
    auth.authenticate_user()
    credentials, project = default()
    gc = gspread.authorize(credentials)

    # Create new spreadsheet
    output_document = gc.create('Solving Business Problems with AI - Output')

    # Create required sheets
    email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
    email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

    order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
    order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

    order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
    order_response_sheet.update([['email ID', 'response']], 'A1:B1')

    inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
    inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

    # Share the spreadsheet publicly
    output_document.share('', perm_type='anyone', role='reader')

    return output_document, email_classification_sheet

def main():
    # Read data
    document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'

    def read_data_frame(sheet_name):
        export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        return pd.read_csv(export_link)

    # Read emails data
    emails_df = read_data_frame('emails')

    # Initialize classifier
    classifier = EmailClassifier(
        api_key='<openai_api_key>',
        base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/'
    )

    # Classify emails
    result_df = classifier.classify_emails(emails_df)

    # Set up Google Sheets and save results
    try:
        output_document, email_classification_sheet = setup_google_sheets()

        # Save classification results to Google Sheet
        set_with_dataframe(email_classification_sheet, result_df)

        # Print the shareable link
        print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")
        print("Classification results have been saved to Google Sheets.")

    except Exception as e:
        print(f"Error saving to Google Sheets: {str(e)}")
        print("Saving results locally instead...")
        result_df.to_csv('email_classification_results.csv', index=False)
        print("Results saved to email_classification_results.csv")

if __name__ == "__main__":
    main()

Shareable link: https://docs.google.com/spreadsheets/d/1BYevlGdI3ho9BMoW7DgVe9xSDbydvH55lc3WawGVA04
Classification results have been saved to Google Sheets.


# Process order requests

In [ ]:
class OrderProcessor:
    def __init__(self, api_key: str, base_url: str):
        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key
        )
        self.products_df = None
        self.order_status = []
        self.order_responses = []

    def load_products(self, products_df: pd.DataFrame):
        """Load and store products data."""
        self.products_df = products_df

    def _extract_product_info(self, message: str) -> List[Dict]:
        """Extract product IDs and quantities using advanced prompt engineering."""
        prompt = f"""You are a product information extractor. Your task is to analyze the customer message and extract product information in a specific JSON format.

        Customer Message:
        {message}

        Available Products:
        {self.products_df[['product_id', 'name', 'category', 'description']].to_json(orient='records', indent=2)}

        Rules:
        1. Extract product IDs (format: 3 letters + 4 numbers, e.g., RSG8901)
        2. Extract quantities in various formats:
          - Numbers (1, 2, 3)
          - Words (a, one, two, three)
          - Phrases (a pair, a couple)
          - Special words (all, remaining, rest)
        3. If quantity is not specified, use 1
        4. If 'all' or 'all the remaining' is specified, use the available stock
        5. Match product descriptions to product IDs when exact IDs are not provided

        You MUST respond with a valid JSON object in this exact format:
        {{
            "products": [
                {{
                    "product_id": "string (required)",
                    "quantity": number (required),
                    "confidence": number between 0 and 1 (required)
                }}
            ]
        }}

        Example response:
        {{
            "products": [
                {{
                    "product_id": "RSG8901",
                    "quantity": 2,
                    "confidence": 0.95
                }}
            ]
        }}"""

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": "You are a product information extractor that always returns valid JSON."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.1
                )

                content = response.choices[0].message.content.strip()
                if not content:
                    print(f"Empty response from GPT (attempt {attempt + 1}/{max_retries})")
                    continue

                # Try to find JSON in the response
                json_start = content.find('{')
                json_end = content.rfind('}') + 1
                if json_start >= 0 and json_end > json_start:
                    content = content[json_start:json_end]

                result = json.loads(content)
                if not isinstance(result, dict) or 'products' not in result:
                    print(f"Invalid JSON structure (attempt {attempt + 1}/{max_retries})")
                    continue

                # Validate each product entry
                valid_products = []
                for product in result['products']:
                    if not all(k in product for k in ['product_id', 'quantity', 'confidence']):
                        continue
                    if not isinstance(product['quantity'], (int, float)):
                        continue
                    if not isinstance(product['confidence'], (int, float)):
                        continue
                    if not 0 <= product['confidence'] <= 1:
                        continue
                    valid_products.append(product)

                if valid_products:
                    return valid_products

                print(f"No valid products found in response (attempt {attempt + 1}/{max_retries})")

            except (json.JSONDecodeError, KeyError, AttributeError) as e:
                print(f"Error in GPT extraction (attempt {attempt + 1}/{max_retries}): {str(e)}")
                if attempt == max_retries - 1:
                    print("All retry attempts failed, falling back to regex extraction")
                    return self._fallback_extract_product_info(message)
                continue

        return self._fallback_extract_product_info(message)

    def _fallback_extract_product_info(self, message: str) -> List[Dict]:
        """Fallback method using regex for product extraction."""
        try:
            # Look for product IDs
            product_pattern = r'[A-Z]{3}\d{4}'
            product_ids = re.findall(product_pattern, message)

            # Look for quantities
            quantity_pattern = r'(?:^|\s)(\d+|\b(?:a|one|two|three|four|five|six|seven|eight|nine|ten|all|remaining|rest)\b)(?:\s*(?:pairs?|pieces?|units?))?'
            quantities = re.findall(quantity_pattern, message.lower())

            # Convert word quantities to numbers
            word_to_num = {
                'a': 1, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
                'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
            }

            product_info = []
            for i, pid in enumerate(product_ids):
                try:
                    if i < len(quantities):
                        qty = quantities[i][0]
                        if qty in word_to_num:
                            quantity = word_to_num[qty]
                        elif qty in ['all', 'remaining', 'rest']:
                            # Get available stock
                            product = self.products_df[self.products_df['product_id'] == pid]
                            quantity = product.iloc[0]['stock'] if not product.empty else 1
                        else:
                            try:
                                quantity = int(qty)
                            except ValueError:
                                # If conversion fails, default to 1
                                quantity = 1
                    else:
                        quantity = 1

                    product_info.append({
                        'product_id': pid,
                        'quantity': quantity,
                        'confidence': 1.0
                    })
                except Exception as e:
                    print(f"Error processing product {pid}: {str(e)}")
                    # Continue with next product if there's an error
                    continue

            return product_info

        except Exception as e:
            print(f"Error in fallback extraction: {str(e)}")
            return []

    def _check_stock(self, product_id: str, quantity: int) -> Tuple[bool, int]:
        """Check if requested quantity is available in stock."""
        try:
            product = self.products_df[self.products_df['product_id'] == product_id]
            if product.empty:
                print(f"Product {product_id} not found in catalog")
                return False, 0

            current_stock = product.iloc[0]['stock']
            return current_stock >= quantity, current_stock

        except Exception as e:
            print(f"Error checking stock for product {product_id}: {str(e)}")
            return False, 0

    def _find_similar_products(self, product_id: str, category: str, limit: int = 3) -> List[Dict]:
        """Find similar products in the same category."""
        category_products = self.products_df[
            (self.products_df['category'] == category) &
            (self.products_df['product_id'] != product_id) &
            (self.products_df['stock'] > 0)
        ]

        if category_products.empty:
            return []

        # Get top N products by stock availability
        return category_products.nlargest(limit, 'stock').to_dict('records')

    def _generate_order_response(self, email_id: str, order_status: List[Dict]) -> str:
        """Generate a response email based on order processing results."""
        # Get product details for the order
        order_details = []
        for item in order_status:
            product = self.products_df[self.products_df['product_id'] == item['product_id']].iloc[0]
            order_details.append({
                'product_id': item['product_id'],
                'name': product['name'],
                'quantity': item['quantity'],
                'price': product['price'],
                'status': item['status'],
                'category': product['category']
            })

        # Find alternatives for out-of-stock items
        alternatives = {}
        for item in order_details:
            if item['status'] == 'out of stock':
                alternatives[item['product_id']] = self._find_similar_products(
                    item['product_id'],
                    item['category']
                )

        prompt = f"""You are a customer service representative for a fashion retail business. Generate a professional response email based on the following order processing results:

        Email ID: {email_id}
        Order Details:
        {json.dumps(order_details, indent=2)}

        Alternative Products:
        {json.dumps(alternatives, indent=2)}

        Guidelines:
        1. If all items are in stock:
          - Confirm the order with enthusiasm
          - List each item with its name, quantity, and price
          - Calculate and show the total price
          - Include next steps (shipping, etc.)
          - Add a personal touch

        2. If some items are out of stock:
          - Acknowledge the order professionally
          - List available items with details
          - List out-of-stock items
          - Suggest specific alternatives from the provided list
          - Explain why the alternatives are good matches
          - Offer to notify when back in stock
          - Be empathetic and helpful

        3. If all items are out of stock:
          - Express regret professionally
          - List the requested items
          - Suggest specific alternatives from the provided list
          - Explain why the alternatives are good matches
          - Offer to notify when back in stock
          - Include a special offer or discount if appropriate

        Please generate a professional, friendly response that addresses the customer's needs. Include specific product details and prices."""

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a professional customer service representative."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return response.choices[0].message.content.strip()

    def process_order(self, email_id: str, message: str) -> Tuple[List[Dict], str]:
        """Process an order request and generate a response."""
        # Extract product information
        product_info = self._extract_product_info(message)

        # Process each product
        order_status = []
        for product in product_info:
            pid = product['product_id']
            quantity = product['quantity']

            # Check stock
            in_stock, current_stock = self._check_stock(pid, quantity)

            # Update order status
            status = "created" if in_stock else "out of stock"
            order_status.append({
                'email_id': email_id,
                'product_id': pid,
                'quantity': quantity,
                'status': status
            })

            # Update stock if order created
            if in_stock:
                self.products_df.loc[self.products_df['product_id'] == pid, 'stock'] -= quantity

        # Generate response
        response = self._generate_order_response(email_id, order_status)

        return order_status, response

    def process_orders(self, emails_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Process all order requests in the dataframe."""
        for _, row in emails_df.iterrows():
            email_id = row['email_id']
            message = row['message']

            # Process order and get status and response
            order_status, response = self.process_order(email_id, message)

            # Store results
            self.order_status.extend(order_status)
            self.order_responses.append({
                'email_id': email_id,
                'response': response
            })

        # Create result dataframes
        order_status_df = pd.DataFrame(self.order_status)
        order_response_df = pd.DataFrame(self.order_responses)

        return order_status_df, order_response_df

def main():
    # Read data
    document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'

    def read_data_frame(sheet_name):
        export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        return pd.read_csv(export_link)

    # Read data
    products_df = read_data_frame('products')
    emails_df = read_data_frame('emails')

    # Filter only order request emails
    order_emails_df = emails_df[emails_df['email_id'].isin(
        pd.read_csv('email_classification_results.csv')[
            pd.read_csv('email_classification_results.csv')['category'] == 'order request'
        ]['email ID']
    )]

    # Initialize processor
    processor = OrderProcessor(
        api_key='<openai_api_key>',
        base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/'
    )

    # Load products
    processor.load_products(products_df)

    # Process orders
    order_status_df, order_response_df = processor.process_orders(order_emails_df)

    # Save results to Google Sheets
    try:
        # Authenticate with Google
        auth.authenticate_user()
        credentials, project = default()
        gc = gspread.authorize(credentials)

        # Get the output document
        output_document = gc.open('Solving Business Problems with AI - Output')

        # Update order-status sheet
        order_status_sheet = output_document.worksheet('order-status')
        set_with_dataframe(order_status_sheet, order_status_df)

        # Update order-response sheet
        order_response_sheet = output_document.worksheet('order-response')
        set_with_dataframe(order_response_sheet, order_response_df)

        print("Order processing results have been saved to Google Sheets.")

    except Exception as e:
        print(f"Error saving to Google Sheets: {str(e)}")
        print("Saving results locally instead...")
        order_status_df.to_csv('order_status_results.csv', index=False)
        order_response_df.to_csv('order_response_results.csv', index=False)
        print("Results saved to local CSV files")

if __name__ == "__main__":
    main()

Order processing results have been saved to Google Sheets.


# Handle Product Inquiry

In [ ]:
class InquiryProcessor:
    def __init__(self, api_key: str, base_url: str):
        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key
        )
        self.products_df = None
        self.vectorizer = None
        self.product_vectors = None
        self.inquiry_responses = []

    def load_products(self, products_df: pd.DataFrame):
        """Load and store products data, prepare for efficient search."""
        self.products_df = products_df

        # Prepare product descriptions for vectorization
        product_descriptions = products_df.apply(
            lambda x: f"{x['name']} {x['category']} {x['description']}",
            axis=1
        ).tolist()

        # Initialize TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            stop_words='english',
            max_features=1000,
            ngram_range=(1, 2)
        )

        # Create product vectors
        self.product_vectors = self.vectorizer.fit_transform(product_descriptions)

    def _find_relevant_products(self, inquiry: str, top_k: int = 3) -> List[Dict]:
        """Find most relevant products using TF-IDF and cosine similarity."""
        # Vectorize the inquiry
        inquiry_vector = self.vectorizer.transform([inquiry])

        # Calculate similarity scores
        similarity_scores = cosine_similarity(inquiry_vector, self.product_vectors).flatten()

        # Get top k products
        top_indices = similarity_scores.argsort()[-top_k:][::-1]

        relevant_products = []
        for idx in top_indices:
            product = self.products_df.iloc[idx]
            relevant_products.append({
                'product_id': product['product_id'],
                'name': product['name'],
                'category': product['category'],
                'description': product['description'],
                'price': product['price'],
                'stock': product['stock'],
                'seasons': product['seasons'],
                'similarity_score': float(similarity_scores[idx])
            })

        return relevant_products

    def _generate_inquiry_response(self, email_id: str, inquiry: str, relevant_products: List[Dict]) -> str:
        """Generate a response to the product inquiry using GPT-4."""
        # Prepare product information for the prompt
        product_info = []
        for product in relevant_products:
            if product['similarity_score'] > 0.1:  # Only include products with meaningful similarity
                product_info.append({
                    'name': str(product['name']),
                    'category': str(product['category']),
                    'description': str(product['description']),
                    'price': float(product['price']),
                    'stock': int(product['stock']),
                    'seasons': str(product['seasons'])
                })

        prompt = f"""You are a knowledgeable fashion retail assistant. Generate a helpful response to the customer's inquiry about products.

        Customer Inquiry:
        {inquiry}

        Relevant Products:
        {json.dumps(product_info, indent=2)}

        Guidelines:
        1. Address the customer's specific questions and concerns
        2. Provide detailed information about relevant products
        3. Include pricing, availability, and seasonal information
        4. Suggest alternatives if appropriate
        5. Be professional, friendly, and helpful
        6. If no products match the inquiry well, explain why and suggest general categories
        7. Include specific product recommendations with reasons why they might be suitable

        Please generate a comprehensive, helpful response that directly addresses the customer's needs."""

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a professional fashion retail assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return response.choices[0].message.content.strip()

    def process_inquiry(self, email_id: str, inquiry: str) -> str:
        """Process a product inquiry and generate a response."""
        # Find relevant products
        relevant_products = self._find_relevant_products(inquiry)

        # Generate response
        response = self._generate_inquiry_response(email_id, inquiry, relevant_products)

        return response

    def process_inquiries(self, emails_df: pd.DataFrame) -> pd.DataFrame:
        """Process all product inquiries in the dataframe."""
        for _, row in emails_df.iterrows():
            email_id = row['email_id']
            message = row['message']

            # Process inquiry and get response
            response = self.process_inquiry(email_id, message)

            # Store result
            self.inquiry_responses.append({
                'email_id': email_id,
                'response': response
            })

        # Create result dataframe
        inquiry_response_df = pd.DataFrame(self.inquiry_responses)

        return inquiry_response_df

def main():
    # Read data
    document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'

    def read_data_frame(sheet_name):
        export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        return pd.read_csv(export_link)

    # Read data
    products_df = read_data_frame('products')
    emails_df = read_data_frame('emails')

    # Filter only product inquiry emails
    inquiry_emails_df = emails_df[emails_df['email_id'].isin(
        pd.read_csv('email_classification_results.csv')[
            pd.read_csv('email_classification_results.csv')['category'] == 'product inquiry'
        ]['email ID']
    )]

    # Initialize processor
    processor = InquiryProcessor(
        api_key='<openai_api_key>',
        base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/'
    )

    # Load products
    processor.load_products(products_df)

    # Process inquiries
    inquiry_response_df = processor.process_inquiries(inquiry_emails_df)

    # Save results to Google Sheets
    try:
        # Authenticate with Google
        auth.authenticate_user()
        credentials, project = default()
        gc = gspread.authorize(credentials)

        # Get the output document
        output_document = gc.open('Solving Business Problems with AI - Output')

        # Update inquiry-response sheet
        inquiry_response_sheet = output_document.worksheet('inquiry-response')
        set_with_dataframe(inquiry_response_sheet, inquiry_response_df)

        print("Inquiry processing results have been saved to Google Sheets.")

    except Exception as e:
        print(f"Error saving to Google Sheets: {str(e)}")
        print("Saving results locally instead...")
        inquiry_response_df.to_csv('inquiry_response_results.csv', index=False)
        print("Results saved to local CSV file")

if __name__ == "__main__":
    main()

Inquiry processing results have been saved to Google Sheets.
